In [57]:
! pip install numpy pandas scikit-learn catboost matplotlib seaborn phik nltk wordcloud imblearn tensorflow

## Поменять

✅Ты делаешь токенизацию текстов, во первых хочется видеть более детальный анализ токенов (помимо облака слов еще диаграммы), проверка пунктуации. Во вторых токенизация для EDA и для модели должна быть одинаковая, иначе анализируешь ты одно, а на вход модели передается другое совсем, в чем нет смысла

✅Также токенизатор должен корректно работать с oov_token

✅Проверь что везде SEED фиксированный используется, чтобы нейронки обучались всегда одинаково для одних и тех же данных

✅Длина последовательностей = максимуму. Максимум часто определяется редкими хвостами → перерасход вычислений и регуляризации. Возьми, например, 95-й перцентиль длины

❓Нет стратификации при сплите train_test_split без stratify=y. Даже при балансе классов это повышает дисперсию метрик.

✅В первой LSTM у тебя Embedding(..., trainable=False) без загруженной матрицы — фактически «замороженная случайная» эмбеддинг-матрица. В этом нет смысла. Либо ты обучаешь эмбеддинги, либо используешь предобученные

✅Круто что ты нашел такую зависимость между 'subject' и 'target', для этого как раз графики и строятся, чтобы такое вот находить. В этом случае нам мл модель вообще не нужна, мы можем таргет предсказывать просто по subject и все, правильно что выкинул

✅Кажется, у тебя утечка данных при использовании 
X_train_text = vectorizer.fit_transform(X_train['text'].fillna(''))
X_test_text = vectorizer.fit_transform(X_test['text'].fillna(''))
CountVectorizer должен быть частью пайплайна

✅В реальных задачах ты никогда не будешь выкидывать текст статьи и оставлять только заголовок, поэтому используй сразу оба столбца

✅У каждой модели хотелось бы посмотреть важности признаков, а именно токенов, ведь это самое главное в текстовой классификации: увидеть какие слова влияют в какую сторону.

✅Добавь еще f1 метрику

✅Тюнинг гиперпараметров советую потренировать на нейронках, там их как раз много. Будет долго обучаться, но для тренировки стоит сделать

❓Еще общее замечание: код очень хаотичный и трудно разобрать, что зачем происходит. Старайся делить его на блоки и описывать что за чем происходит и логику своих действий, почему ты делаешь то или иное действие

✅Еще тебе нужно разбить данные по времени, то есть в test самые свежие новости, в train более ранние

✅Еще вспомнил, попробуй CNN использовать. Эта штука очень быстро обучается и точность высокая

____________________________________________________________________________________________________________________________________________________________________________________________________

✅Почему-то эта разбивка есть в improvements, но не в бейслайне. Сравнивать модели нужно на одной и той же выборке

❓TextPreprocessor - тебе не нужен в пайплайне, можешь просто метод создать и применить ко всему датасету
KerasTokenizerTransformer тоже не особо понял зачем, почему нельзя обычный Tokenizer добавить в пайплайн?

✅Не вижу смысла в двух слоях LSTM и доп Dense слое. Попробуй минимальный сетап, посмотри на метрики, пробуй улучшать, оставляй только нужное. Должен быть компромисс между качеством и скоростью обучения. То же самое с RNN. C CNN тоже слишком громоздкая архитектура получилась

## Начало

In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from helper import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from catboost import CatBoostClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from nltk.corpus import stopwords
from plots import *
import re

In [59]:
SEED = 42

In [60]:
stop_words = set(stopwords.words('english'))

In [61]:

def split_dataset(path, target_value, date_col='date', target_col='target', ratio=0.8): # т.к. датасет лежит в 2 файлах и надо в тест взять самые новые новости приходится выкручиваться
    df = pd.read_csv(path)
    df[target_col] = target_value  # ставим таргет
    df = df.sort_values(date_col).reset_index(drop=True)
    split_idx = int(len(df) * ratio)

    train = df.iloc[:split_idx]
    test = df.iloc[split_idx:]

    X_train = train.drop(columns=[target_col])
    y_train = train[target_col]
    X_test = test.drop(columns=[target_col])
    y_test = test[target_col]

    return X_train, X_test, y_train, y_test

# true = класс 1, false = класс 0
X_train_true, X_test_true, y_train_true, y_test_true = split_dataset('old/True.csv', target_value=1) # отдельно сплитим каждый датасет таким образом, что в тест попали самые свежие
X_train_false, X_test_false, y_train_false, y_test_false = split_dataset('old/Fake.csv', target_value=0)

# объединяем
X_train = pd.concat([X_train_true, X_train_false], ignore_index=True)
y_train = pd.concat([y_train_true, y_train_false], ignore_index=True)
X_test = pd.concat([X_test_true, X_test_false], ignore_index=True)
y_test = pd.concat([y_test_true, y_test_false], ignore_index=True)

# X_train['combo'] = X_train['title'] + X_train['text']
# X_test['combo'] = X_test['title'] + X_test['text']

# финальное распределение
print('Train distribution:\n', y_train.value_counts(normalize=True))
print('Test distribution:\n', y_test.value_counts(normalize=True))

Train distribution:
 target
0    0.522984
1    0.477016
Name: proportion, dtype: float64
Test distribution:
 target
0    0.522993
1    0.477007
Name: proportion, dtype: float64


In [62]:
X_train.head(10)

,title,text,subject,date
0,Trump sued in Kentucky by three who say he inc...,"LOUISVILLE, Ky. (Reuters) - Three people who c...",politicsNews,"April 1, 2016"
1,"In record year for political ads, media buyers...",(Reuters) - Media buyers expect record politic...,politicsNews,"April 1, 2016"
2,"Obama, Republicans urge Trump to soften tone",WASHINGTON (Reuters) - Donald Trump is facing ...,politicsNews,"April 1, 2016"
3,State Department says halts review of Clinton ...,WASHINGTON (Reuters) - The U.S. State Departme...,politicsNews,"April 1, 2016"
4,Obama: Trump doesn't know much about foreign p...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"April 1, 2016"
5,Obama says 'no doubt' civilians have been kill...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"April 1, 2016"
6,Former New York deputy secretary of state accu...,NEW YORK (Reuters) - A former New York deputy ...,politicsNews,"April 1, 2016"
7,Hillary Clinton to propose $10 billion manufac...,WASHINGTON (Reuters) - Democratic presidential...,politicsNews,"April 1, 2016"
8,Trump's probability of winning nomination drop...,NEW YORK (Reuters) - The probability that Repu...,politicsNews,"April 1, 2016"
9,Senator Warren seeks investigation of insurers...,WASHINGTON (Reuters) - Senator Elizabeth Warre...,politicsNews,"April 1, 2016"


In [63]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=SEED, shuffle=True, stratify=None)

# X_train.drop(columns=['subject', 'date'], inplace=True)
# X_test.drop(columns=['subject', 'date'], inplace=True)
# X_val.drop(columns=['subject', 'date'], inplace=True)

X_train.drop(columns=['subject', 'date', 'text'], inplace=True) # удаляем неинформативные колонки
X_test.drop(columns=['subject', 'date', 'text'], inplace=True)
X_val.drop(columns=['subject', 'date', 'text'], inplace=True)

X_train.head(10)

,title
0,Trump sued in Kentucky by three who say he inc...
1,"In record year for political ads, media buyers..."
2,"Obama, Republicans urge Trump to soften tone"
3,State Department says halts review of Clinton ...
4,Obama: Trump doesn't know much about foreign p...
5,Obama says 'no doubt' civilians have been kill...
6,Former New York deputy secretary of state accu...
7,Hillary Clinton to propose $10 billion manufac...
8,Trump's probability of winning nomination drop...
9,Senator Warren seeks investigation of insurers...


In [64]:
len(X_val), len(X_test)

(4490, 4491)

блок для тестов

In [65]:
# def preprocessing(text):
#      text = text.lower()
#      text = re.sub('[^a-zA-Z]', ' ', text)
#      text = re.sub('\s+', ' ', text)
#      text = re.sub(r'https?://\S+|www\.\S+|[^a-zA-Z\s]', '', text)
#      text = re.sub(r'<.*?>', '', text)
#      return text 

# vectorized = np.vectorize(preprocessing)
# x_train = vectorized(X_train)
# x_test = vectorized(X_test)
# x_val = vectorized(X_val)

# print('the first 20 words of first train text after preprocessing:')
# visualize = ''
# counter = 0 

# for i in x_train[0].split(' '):
#     visualize = visualize + i + " " 
#     if counter>20:
#         break
#     counter = counter + 1 

# print(visualize)

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(x_train)

# train_seq = tokenizer.texts_to_sequences(x_train)
# test_seq = tokenizer.texts_to_sequences(x_test)
# val_seq = tokenizer.texts_to_sequences(x_val)
# vocab_size = len(tokenizer.word_index) + 1 
# max_length = max(len(sequence) for sequence in train_seq)

# train_seq = pad_sequences(train_seq, maxlen = max_length, padding = 'post', truncating = 'post')
# test_seq = pad_sequences(test_seq, maxlen = max_length, padding = 'post', truncating = 'post')
# val_seq = pad_sequences(val_seq, maxlen = max_length, padding = 'post', truncating = 'post')

## Preprocessing

In [66]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nemow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Я посмотрел, обычный керас tokenizer нельзя использовать без кастомных классов, т.к. у него нет своих методов .fit(), .transform()

Я пытался сделать через FunctionTransformer, но:

- Ему все равно надо прописывать .fit(), .transform()

- Он в запоминает только OVV токен

Поэтому оставил как есть, очистка текста тоже осталась в пайплайне. т.к. при инференсе все равно пришлось бы текст точно так же чистить 

In [67]:
from pipeline import CustomPipeline
import nltk


nltk.download('stopwords')
from nltk.corpus import stopwords

pipeline = CustomPipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nemow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
train_seq = pipeline.fit_transform(X_train) # применяем пайплайн
val_seq = pipeline.transform(X_val)
test_seq = pipeline.transform(X_test)

vocab_size = pipeline.named_steps['tokenizer'].vocab_size
max_length = pipeline.named_steps['tokenizer'].max_length

vocab_size, max_length

(18616, 35)

In [69]:
train_seq

array([[   2, 2638, 2548, ...,    0,    0,    0],
       [ 871,  105,  151, ...,    0,    0,    0],
       [   6,   25,  872, ...,    0,    0,    0],
       ...,
       [  31,  231, 6390, ...,    0,    0,    0],
       [  23,   86, 3744, ...,    0,    0,    0],
       [ 157,   31,   19, ...,    0,    0,    0]],
      shape=(35917, 35), dtype=int32)

## Тестовая секция с Time Series

In [70]:
df = pd.read_csv('dataset.csv', index_col=0)

df['date'] = pd.to_datetime(df['date'], format='mixed', dayfirst=True, errors='coerce')

df_sorted = df.sort_values(by='date')

df_sorted

,title,text,subject,date,target
15885,FLASHBACK: KING OBAMA COMMUTES SENTENCES OF 22...,Just making room for Hillary President Obama t...,politics,2015-03-31,0
15890,HILLARY RODHAM NIXON: A CANDIDATE WITH MORE BA...,The irony here isn t lost on us. Hillary is be...,politics,2015-03-31,0
15889,BENGHAZI PANEL CALLS HILLARY TO TESTIFY UNDER ...,Does anyone really think Hillary Clinton will ...,politics,2015-03-31,0
15888,OH NO! GUESS WHO FUNDED THE SHRINE TO TED KENNEDY,Nothing like political cronyism to make your s...,politics,2015-03-31,0
15887,WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...,"In case you missed it Sen. Harry Reid (R-NV), ...",politics,2015-03-31,0
...,...,...,...,...,...
17432,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,Government News,NaT,0
17433,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,Government News,NaT,0
18933,Homepage,[vc_row][vc_column width= 1/1 ][td_block_trend...,left-news,NaT,0
21869,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,left-news,NaT,0


In [71]:
X = df_sorted.drop(columns=['target', 'subject', 'date'])
# X = df_sorted.drop(columns=['target', 'subject', 'date', 'text'])
y = df_sorted['target']

## LSTM

сделаем простую lstm модель

In [72]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import warnings


warnings.filterwarnings('ignore')

In [73]:
epochs = 10
lr = 1e-3
embedding_dim = 10

model = Sequential([
    Input(shape = (max_length,)),
    Embedding(vocab_size, embedding_dim, input_length = max_length, trainable = True), # установка trainable = True дала 2% 
    
    Bidirectional(LSTM(128)),

    Dropout(0.2),
    
    Dense(1, activation = 'sigmoid')
])
 
model.compile(optimizer = Adam(learning_rate = lr), loss = BinaryCrossentropy(), metrics = ['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 35, 10)         │       186,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       142,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 328,753 (1.25 MB)

 Trainable params: 328,753 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

In [74]:
tf.config.list_physical_devices('GPU')

[]

In [75]:
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

In [76]:
tscv = TimeSeriesSplit(n_splits=3)
metrics_all = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    if y_train.nunique() < 2 or y_test.nunique() < 2:
        print(f"Skipping fold {fold}: not enough class variety")
        continue

    # делим test ещё на val/test (50/50)
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test, 
        test_size=0.5, random_state=SEED, shuffle=True
    )

    train_seq = pipeline.transform(X_train)
    val_seq = pipeline.transform(X_val)
    test_seq = pipeline.transform(X_test)


    model_es = EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)
    model_rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='min')

    model.fit(
        train_seq, y_train,
        epochs=epochs,
        validation_data=(val_seq, y_val),
        callbacks=[model_es, model_rlr],
        verbose=0
    )

    y_pred_test = np.round(model.predict(test_seq, verbose=0))

    f1 = f1_score(y_test, y_pred_test)

    metrics = {
        'fold': fold,
        'F1': f1,
        'Test Accuracy': accuracy_score(y_test, y_pred_test)
    }

    metrics_all.append(metrics)

    y_pred_train = np.round(model.predict(train_seq, verbose = 0))

    print(f'Fold: {fold}')
    print(f'Train Accuracy : {accuracy_score(y_train, y_pred_train) * 100:.2f} %')
    print(f'Test Accuracy  : {accuracy_score(y_test, y_pred_test) * 100:.2f} %')
    print('_'*40)

df_metrics = pd.DataFrame(metrics_all)
print(df_metrics)
print("\nСредний F1 по фолдам:", df_metrics['F1'].mean())

Fold: 0
Train Accuracy : 99.97 %
Test Accuracy  : 99.52 %
________________________________________
Fold: 1
Train Accuracy : 99.81 %
Test Accuracy  : 99.38 %
________________________________________
Fold: 2
Train Accuracy : 99.73 %
Test Accuracy  : 98.65 %
________________________________________
   fold        F1  Test Accuracy
0     0  0.992548       0.995189
1     1  0.993854       0.993763
2     2  0.991880       0.986458

Средний F1 по фолдам: 0.9927607374924499


точность примерно такая же как у классической модели поверх TF-IDF 

## SimpleRNN

In [77]:
from tensorflow.keras.layers import SimpleRNN

epochs = 10
lr = 1e-3
embedding_dim = 10

model = Sequential([
    Input(shape=(max_length,)),
    Embedding(vocab_size, embedding_dim, input_length=max_length, trainable=True),

    Bidirectional(SimpleRNN(128)),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])
 
model.compile(optimizer = Adam(learning_rate = lr), loss = BinaryCrossentropy(), metrics = ['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 35, 10)         │       186,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │        35,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,001 (867.19 KB)

 Trainable params: 222,001 (867.19 KB)

 Non-trainable params: 0 (0.00 B)

In [78]:
tscv = TimeSeriesSplit(n_splits=3)
metrics_all = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    if y_train.nunique() < 2 or y_test.nunique() < 2:
        print(f"Skipping fold {fold}: not enough class variety")
        continue

    # делим test ещё на val/test (50/50)
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test, 
        test_size=0.5, random_state=SEED, shuffle=True
    )

    train_seq = pipeline.transform(X_train)
    val_seq = pipeline.transform(X_val)
    test_seq = pipeline.transform(X_test)


    model_es = EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)
    model_rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='min')

    model.fit(
        train_seq, y_train,
        epochs=epochs,
        validation_data=(val_seq, y_val),
        callbacks=[model_es, model_rlr],
        verbose=0
    )

    y_pred_test = np.round(model.predict(test_seq, verbose=0))

    f1 = f1_score(y_test, y_pred_test)

    metrics = {
        'fold': fold,
        'F1': f1,
        'Test Accuracy': accuracy_score(y_test, y_pred_test)
    }

    metrics_all.append(metrics)

    y_pred_train = np.round(model.predict(train_seq, verbose = 0))

    print(f'Fold: {fold}')
    print(f'Train Accuracy : {accuracy_score(y_train, y_pred_train) * 100:.2f} %')
    print(f'Test Accuracy  : {accuracy_score(y_test, y_pred_test) * 100:.2f} %')
    print('_'*40)

df_metrics = pd.DataFrame(metrics_all)
print(df_metrics)
print("\nСредний F1 по фолдам:", df_metrics['F1'].mean())

Fold: 0
Train Accuracy : 99.98 %
Test Accuracy  : 99.71 %
________________________________________
Fold: 1
Train Accuracy : 99.91 %
Test Accuracy  : 99.84 %
________________________________________
Fold: 2
Train Accuracy : 99.88 %
Test Accuracy  : 99.82 %
________________________________________
   fold        F1  Test Accuracy
0     0  0.995597       0.997149
1     1  0.998425       0.998396
2     2  0.998938       0.998218

Средний F1 по фолдам: 0.9976534041865911


на 1-2% хуже LSTM, но обучаемых параметров в 5 раз меньше

## CNN

In [79]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout, Dense


epochs = 10
lr = 1e-3
embedding_dim = 10

inp = Input(shape=(max_length,))
x = Embedding(vocab_size, embedding_dim, input_length=max_length, trainable=True)(inp)

convs = [GlobalMaxPooling1D()(Conv1D(128, k, activation='relu')(x)) for k in [3, 4, 5]]

x = Concatenate()(convs)
x = Dense(256, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=out)

model.compile(optimizer=Adam(learning_rate=lr), loss=BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 35)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 35, 10)    │    186,160 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 33, 128)   │      3,968 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 32, 128)   │      5,248 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 31, 128)   │      6,528 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_4[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_5[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     98,560 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        257 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 300,721 (1.15 MB)

 Trainable params: 300,721 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
tscv = TimeSeriesSplit(n_splits=3)
metrics_all = []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    if y_train.nunique() < 2 or y_test.nunique() < 2:
        print(f"Skipping fold {fold}: not enough class variety")
        continue

    # делим test ещё на val/test (50/50)
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test, 
        test_size=0.5, random_state=SEED, shuffle=True
    )

    train_seq = pipeline.transform(X_train)
    val_seq = pipeline.transform(X_val)
    test_seq = pipeline.transform(X_test)


    model_es = EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)
    model_rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='min')

    model.fit(
        train_seq, y_train,
        epochs=epochs,
        validation_data=(val_seq, y_val),
        callbacks=[model_es, model_rlr],
        verbose=0
    )

    y_pred_test = np.round(model.predict(test_seq, verbose=0))

    f1 = f1_score(y_test, y_pred_test)

    metrics = {
        'fold': fold,
        'F1': f1,
        'Test Accuracy': accuracy_score(y_test, y_pred_test)
    }

    metrics_all.append(metrics)

    y_pred_train = np.round(model.predict(train_seq, verbose = 0))

    print(f'Fold: {fold}')
    print(f'Train Accuracy : {accuracy_score(y_train, y_pred_train) * 100:.2f} %')
    print(f'Test Accuracy  : {accuracy_score(y_test, y_pred_test) * 100:.2f} %')
    print('_'*40)

df_metrics = pd.DataFrame(metrics_all)
print(df_metrics)
print("\nСредний F1 по фолдам:", df_metrics['F1'].mean())

Fold: 0
Train Accuracy : 100.00 %
Test Accuracy  : 99.75 %
________________________________________
Fold: 1
Train Accuracy : 99.91 %
Test Accuracy  : 99.80 %
________________________________________
Fold: 2
Train Accuracy : 99.89 %
Test Accuracy  : 99.71 %
________________________________________
   fold        F1  Test Accuracy
0     0  0.996150       0.997505
1     1  0.998075       0.998040
2     2  0.998303       0.997149

Средний F1 по фолдам: 0.9975090363963899
